In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

train_features=np.load("../extracted_features/FS/input_features_free_train.npy")
labels_stress_train=np.load("../extracted_features/FS/label_free_train.npy")


test_features=np.load("../extracted_features/FS/input_features_free_test.npy")
labels_stress_test=np.load("../extracted_features/FS/label_free_test.npy")


val_features=np.load("../extracted_features/FS/input_features_free_val.npy")
labels_stress_val=np.load("../extracted_features/FS/label_free_val.npy")


In [2]:
print(labels_stress_train.shape,labels_stress_train)

(1121,) [0. 0. 0. ... 1. 1. 1.]


In [3]:
batch_size=20
train_data = Dataset(train_features, labels_stress_train)
test_data=Dataset(test_features,labels_stress_test)
val_data=Dataset(val_features,labels_stress_val)
train_data_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [4]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [5]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [6]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 24
        self.hidden_dim = 250
        self.L=64
        self.layer_dim = 3
        self.output_dim=1
        self.fc1 = nn.Sequential(
            nn.Linear(self.hidden_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_dim, self.L),
            nn.Tanh(),
            nn.Linear(self.L, 1)
        )
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))

        A = self.attention(out) 
        A = torch.transpose(A, 2,1)  
        A = F.softmax(A, dim=2) 
#         print(A.shape)
        M = torch.matmul(A, out) .squeeze(1)
#         print(M.shape)
        out=self.fc1(M)
#         print(out.shape)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [7]:
def train(model,train_data_loader):
    best_acc1 = 0
    modelname=[]
    truth=[]
    preds=[]
    model.train()
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))

def evalue(model,best_acc1,val_data_loader,modelname,it):
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()
    best_model="./saved_models/best_model_fs"
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(it)+".pth")
        modelname.append(best_model+'_epoch_'+str(it)+".pth")
        print('Best validation accuracy1 ', best_acc1)
    return best_acc1
    

In [8]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 100
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [9]:
best_acc1=0
modelname=[]
for it in range(epoch+1):
    train(model,train_data_loader)
    best_acc1=evalue(model,best_acc1,val_data_loader,modelname,it)


EPOCH  0
Train : Loss: 0.0345, Train acc1 : 0.5254
Val : Loss: 0.0350, Val acc1 : 0.4316
Best validation accuracy1  0.43157894736842106
EPOCH  1
Train : Loss: 0.0344, Train acc1 : 0.5406
Val : Loss: 0.0353, Val acc1 : 0.4316
Best validation accuracy1  0.43157894736842106
EPOCH  2
Train : Loss: 0.0339, Train acc1 : 0.5584
Val : Loss: 0.0361, Val acc1 : 0.4368
Best validation accuracy1  0.4368421052631579
EPOCH  3
Train : Loss: 0.0332, Train acc1 : 0.5718
Val : Loss: 0.0353, Val acc1 : 0.5316
Best validation accuracy1  0.531578947368421
EPOCH  4
Train : Loss: 0.0322, Train acc1 : 0.5825
Val : Loss: 0.0358, Val acc1 : 0.4895
EPOCH  5
Train : Loss: 0.0310, Train acc1 : 0.6039
Val : Loss: 0.0399, Val acc1 : 0.5368
Best validation accuracy1  0.5368421052631579
EPOCH  6
Train : Loss: 0.0309, Train acc1 : 0.6218
Val : Loss: 0.0379, Val acc1 : 0.5158
EPOCH  7
Train : Loss: 0.0294, Train acc1 : 0.6494
Val : Loss: 0.0412, Val acc1 : 0.5211
EPOCH  8
Train : Loss: 0.0302, Train acc1 : 0.6343
Val : 

EPOCH  89
Train : Loss: 0.0086, Train acc1 : 0.9126
Val : Loss: 0.1364, Val acc1 : 0.3947
EPOCH  90
Train : Loss: 0.0149, Train acc1 : 0.8492
Val : Loss: 0.1209, Val acc1 : 0.4474
EPOCH  91
Train : Loss: 0.0127, Train acc1 : 0.8724
Val : Loss: 0.1093, Val acc1 : 0.4158
EPOCH  92
Train : Loss: 0.0111, Train acc1 : 0.8921
Val : Loss: 0.1391, Val acc1 : 0.3474
EPOCH  93
Train : Loss: 0.0091, Train acc1 : 0.8965
Val : Loss: 0.1383, Val acc1 : 0.3737
EPOCH  94
Train : Loss: 0.0106, Train acc1 : 0.8965
Val : Loss: 0.1296, Val acc1 : 0.4368
EPOCH  95
Train : Loss: 0.0088, Train acc1 : 0.9081
Val : Loss: 0.1306, Val acc1 : 0.3895
EPOCH  96
Train : Loss: 0.0097, Train acc1 : 0.9099
Val : Loss: 0.1433, Val acc1 : 0.3421
EPOCH  97
Train : Loss: 0.0085, Train acc1 : 0.9090
Val : Loss: 0.1460, Val acc1 : 0.4000
EPOCH  98
Train : Loss: 0.0088, Train acc1 : 0.9045
Val : Loss: 0.1619, Val acc1 : 0.3474
EPOCH  99
Train : Loss: 0.0076, Train acc1 : 0.9170
Val : Loss: 0.1574, Val acc1 : 0.3579
EPOCH  100

In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.56 0.57 0.51 0.64 0.57 0.50


In [11]:
for i in range(len(modelname)):
    modeltest=Classifier()
    best_state=torch.load(modelname[i])
    modeltest.load_state_dict(best_state)
    modeltest.cuda()
    modeltest.eval()
    truth=[]
    preds=[]
    for minibatch in test_data_loader:
                X_test, Y1_test  = minibatch
                X_test=X_test.cuda()
                Y1_test=Y1_test.cuda()
                output_test = modeltest(X_test.float())
                output_test=output_test.squeeze(1)
                prediction = torch.ge(output_test, 0.5).float()
                truth.extend(Y1_test.tolist())
                preds.extend(prediction.tolist())
    acc=accuracy_score(truth,preds)
    # print(truth,preds)
    tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
    f1score=f1_score(truth, preds)
    precision=precision_score(truth, preds)
    recall=recall_score(truth,preds)
    roc=roc_auc_score(truth,preds)
    specificity=tn/(tn+fp)

    print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))   

0.45 0.62 0.45 1.00 0.50 0.00
0.45 0.62 0.45 1.00 0.50 0.00
0.44 0.61 0.44 0.97 0.48 0.00
0.46 0.61 0.45 0.92 0.50 0.07
0.56 0.57 0.51 0.64 0.57 0.50
